In [1]:
import sys
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, RectangleSkyRegion
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.data import DataStore
from gammapy.datasets import Datasets, FluxPointsDataset, MapDataset
from gammapy.modeling import Fit
from gammapy.modeling.models import SkyModel, LogParabolaSpectralModel, PointSpatialModel, PowerLawSpectralModel, FoVBackgroundModel, Models
from gammapy.estimators import FluxPoints,FluxPointsEstimator, ExcessMapEstimator
import runmatching_analysis as util

In [5]:
def save(fig, filename):
    savedir = '{}'.format(plotDir)
    for form in ['png', 'pdf']:
        fig.savefig('{}/{}.{}'.format(plotDir, filename, form),dpi=150, bbox_inches = "tight")

# Load the Runlist

In [6]:
# load the data and the background model
# requirement: A runlist with ON runs in the first column, OFF runs in the second and fractional run deviation in the third.
data_path = '/path/to/runlist'
runs = np.loadtxt(data_path, usecols=(0,), dtype=int)
off_runs = np.loadtxt(data_path, usecols=(1,), dtype=int)
deviation = np.loadtxt(data_path, usecols=(2,), dtype=float)

basedir = '/path/to/fits/files'
print(basedir)
ds = DataStore.from_dir(basedir,
                           'hdu-index.fits.gz'.format(bgmodel_version),
                           'obs-index.fits.gz'.format(bgmodel_version))
obs_list = ds.get_observations(runs) 
obs_list_off = ds.get_observations(off_runs)

/home/wecapstor1/caph/shared/hess/fits/fits_data/prod05/hess1/std_ImPACT_fullEnclosure_updated


In [7]:
livetime = []
zenith = []
for i in range(0,len(obs_list)):
    livetime.append(obs_list[i].observation_live_time_duration.value)
    zenith.append(ds.obs_table[ds.obs_table['OBS_ID']==obs_list[i].obs_id]["ZEN_PNT"])

target_names = []
for run in off_runs:
    name = DB_general[DB_general['Run']==run]['Target_Name'].iloc[0]
    if name not in target_names:
        target_names.append(name)

print('Mean fractional run deviation:', np.mean(deviation))
print('Livetime:',np.sum(livetime)/3600)
print('Zenith Angle range:',np.min(zenith), np.max(zenith))
print('The OFF run targets are:', target_names)

Mean fractional run deviation: 0.3496839763066876
Livetime: 1.8727777777777768
Zenith Angle range: 45.368176 48.61021
The OFF run targets are: ['GRB 050209', 'M 87']


# Create the dataset

In [8]:
#define the target geometry of the dataset
ra_obj = 83.6292
dec_obj = 22.0125
name_obj = 'Crab'
target = SkyCoord(ra_obj, dec_obj, frame='icrs', unit='deg')

e_reco = np.logspace(-1, 2, 25) * u.TeV 
e_true = np.logspace(-1, 2, 49) * u.TeV 

energy_axis = MapAxis.from_edges(e_reco, unit='TeV', name='energy', interp='log')
energy_axis_true = MapAxis.from_edges(e_true, unit='TeV', name="energy_true", interp='log')

geom = WcsGeom.create(
    skydir=(ra_obj, dec_obj),
    binsz=0.02,
    width=(7, 7),
    frame="icrs",
    proj="CAR",
    axes=[energy_axis],)

In [73]:
# load the file in which your systematic errors are saved
systematic_shift = pd.read_csv (
    '/path/to/systematic/error/file.csv', sep='\t'
)
systematic_shift = systematic_shift.fillna(0)

In [74]:
dataset_creator = util.matching_dataset(systematic_shift, ds, obs_list, obs_list_off, deviation, geom, energy_axis_true, offset_max=2.0* u.deg)

In [75]:
stacked = dataset_creator.compute_matched_dataset( corrections='all', systematics=None, debug=0)
stacked.write('{}/dataset-crab_corr.fits.gz'.format(outputBase), overwrite=True)

Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23523


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23526


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23559


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23592


In [76]:
stacked_low = dataset_creator.compute_matched_dataset( corrections='all', systematics='low', debug=0)
stacked_low.write('{}/dataset-crab_low.fits.gz'.format(outputBase), overwrite=True)

Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23523


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23526


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23559


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23592


In [77]:
stacked_high = dataset_creator.compute_matched_dataset( corrections='all', systematics='high', debug=0)
stacked_high.write('{}/dataset-crab_high.fits.gz'.format(outputBase), overwrite=True)

Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23523


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23526


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23559


Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'


23592


In [78]:
stacked_fov = dataset_creator.standard_dataset(debug=0)
stacked_fov.write('{}/dataset-crab_fov.fits.gz'.format(outputBase), overwrite=True)

Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'
Missing 'HDUCLAS2' keyword assuming 'BKG'
